1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [121]:
def converter(x, base_i, base_f):
    '''

    Converts x from initial base_i to final base_f
    Parameters:
        x: string if base_i is hex or bin, integer if base_i is dec
        bin numbers should be in the form 0b.. whereas hex numbers 0x..

        base_i: 'dec', 'hex' or 'bin' initial base
        base_f: 'dec', 'hex' or 'bin' final base
        
    '''

    if base_i != 'dec' and base_i != 'bin' and base_i != 'hex':
        raise ValueError ("Invalid initial base")
    if base_f != 'dec' and base_f != 'bin' and base_f != 'hex':
        raise ValueError ("Invalid final base")
        
    # from binary to decimal 
    if base_i == 'bin':
        n_bin = list(x.strip(" ")) # convert to list, i.e bin(10) = 0b1010 -> ['0', 'b', '1', '0', '1', '0']
        n_bin = [int(n_bin[i]) for i in range(2, len(n_bin))] # indices 0/1 are "0b" -> range starts from 2
        n_bin.reverse() 
        n_dec = 0
        i = 0
        for a in n_bin: 
            n_dec += (2**i) * int(a)
            i += 1 

    # from hexadecimal to decimal (same procedure as before)
    elif base_i == 'hex':
        n_hex = list(x.strip(''))

        for i in range(len(n_hex)):
                if n_hex[i] == 'a': n_hex[i] = 10
                if n_hex[i] == 'b': n_hex[i] = 11
                if n_hex[i] == 'c': n_hex[i] = 12
                if n_hex[i] == 'd': n_hex[i] = 13
                if n_hex[i] == 'e': n_hex[i] = 14
                if n_hex[i] == 'f': n_hex[i] = 15

        n_hex = [int(n_hex[i]) for i in range(2, len(n_hex))]
        n_hex.reverse()
        n_dec = 0
        i = 0
        for a in n_hex:
            n_dec += (16**i) * int(a)
            i += 1 

    # base of initial number is already 10
    else: 
        n_dec = x
    
    if base_f == 'dec':
        return n_dec


    # from decimal to bin 
    if base_f == 'bin':  
        r = []
        q = int(n_dec / 2) # [7,3,1,0]
        r.append(n_dec % 2) # We store resto in a list [1,1,1,1]
        while q != 0:
            n_dec = q
            q = int(n_dec / 2)
            r.append(n_dec % 2)
        r.reverse()
        r = ''.join([str(i) for i in r])
    

    elif base_f == 'hex':
        r = []
        q = int(n_dec / 16)     
        r.append(n_dec % 16)      
        while q != 0:
            n_dec = q       
            q = int(n_dec / 16)       
            r.append(n_dec % 16)            
        # Now we have to change numbers from 10 to 16 to a,b,c, etc...
            for i in range(len(r)):
                if r[i] == 10: r[i] = 'a'
                if r[i] == 11: r[i] = 'b'
                if r[i] == 12: r[i] = 'c'
                if r[i] == 13: r[i] = 'd'
                if r[i] == 14: r[i] = 'e'
                if r[i] == 15: r[i] = 'f'

        r.reverse()  
        r = ''.join([str(i) for i in r])  
        
    return r

c = 9999
c_bin = bin(c)
c_hex = hex(c)


print('Using Python functions: initial number is {} in decimal, {} in binary, {} in hexadecimal\n'.format(c,c_bin,c_hex))
print('Using user function:')
print('Converting {} from decimal to binary -> {}'.format(c,converter(c,'dec','bin')))
print('Converting {} from binary to decimal -> {}'.format(c_bin,converter(c_bin,'bin','dec')))
print('Converting {} from decimal to hexadecimal -> {}'.format(c,converter(c,'dec','hex')))
print('Converting {} from hexadecimal to decimal -> {}'.format(c_hex,converter(c_hex,'hex','dec')))
print('Converting {} from hexadecimal to binary -> {}'.format(c_hex,converter(c_hex,'hex','bin')))
print('Converting {} from binary to hexadecimal -> {}'.format(c_bin,converter(c_bin,'bin','hex')))

Using Python functions: initial number is 9999 in decimal, 0b10011100001111 in binary, 0x270f in hexadecimal

Using user function:
Converting 9999 from decimal to binary -> 10011100001111
Converting 0b10011100001111 from binary to decimal -> 9999
Converting 9999 from decimal to hexadecimal -> 270f
Converting 0x270f from hexadecimal to decimal -> 9999
Converting 0x270f from hexadecimal to binary -> 10011100001111
Converting 0b10011100001111 from binary to hexadecimal -> 270f


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [150]:


def bin_to_float(x):
    '''

    Parameters:
        x: 32 bit word in a string
        
    '''
    if not isinstance(x,str):
        raise ValueError ('Input value is not a string')
    
    l = list(x.strip(" "))
    
    if len(l) != 32:
        raise ValueError ('Number of bins is not 32')
         

    sign = ' '

    if l[0] == '0': 
        sign += '+'
    if l[0] == '1': 
        sign += '-' 


    exp = [int(l[i]) for i in range(1, 9)]
    r = ''.join([str(i) for i in exp])
    exp_dec = converter(r,"bin","dec")

    man = [int(l[i]) for i in range(9, len(l))]
     
    s = 0
    for j in range(1,len(man)):
        s += (man[j-1]) * 2 **(-j)

    n = ( 1 +s )* pow(2, exp_dec - 127)

    return(sign + str(n))

word1 = "00000011111000000000000000000000"
word2 = "10000001101001001100000000000000"

x = bin_to_float(word1)
y = bin_to_float(word2)

print('First word is {} -> {}'.format(word1,x))
print('Second word is {} -> {}'.format(word2,y))


First word is 00000011111000000000000000000000 ->  +1.316553672920962e-36
Second word is 10000001101001001100000000000000 ->  -6.051959196811621e-38


In [149]:
def float_to_bin(num):
    bits, = struct.unpack('!I', struct.pack('!f', num))
    return "{:032b}".format(bits)

print(float_to_bin(+1.316553672920962e-36))
print(float_to_bin(-6.051959196811621e-38))

00000011111000000000000000000000
10000001101001001100000000000000


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [151]:
import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

In [40]:
under = 1. 
over = 1. 
N = 2000
i = 0

while i < N:
    o = over
    over = over * 2
    i += 1 
    if over == float('inf'):
        pos_o = i
        break   
    #print(i,over)

j = 0
while j < N:
    u = under
    under = under / 2
    j += 1
    if under == 0. :
        pos_u = j
        break   
    #print(j,under)


print('Overflow limit reached after ' + str(pos_o) + ' iterations and it is equal to: '+ str(o))
print('Underflow limit reached after ' + str(pos_u) + ' iterations and it is equal to: '+ str(u))


Overflow limit reached after 1024 iterations and it is equal to: 8.98846567431158e+307
Underflow limit reached after 1075 iterations and it is equal to: 5e-324


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [165]:
# Addding an increasingly small number to 7
for e in [14, 15, 16]: 
    print (7+1.0*10**-e)

1.0

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
